# **Exploratory Data Analysis**

## **Project Rationale**

**Introduction**
In today’s multimedia-driven world, film posters serve not only as advertisements but also as visual representations of a movie’s theme and genre. Identifying a film’s genre based solely on its poster imagery has practical applications, such as automated indexing on streaming platforms and improving user recommendations.

Our team is undertaking this project to deepen our understanding of practical image classification in real-world scenarios. By applying techniques —ranging from traditional feature extraction methods to modern deep learning approaches—we aim to build a robust classification pipeline capable of handling a challenging multi-genre task.

## **This Notebook**

In this notebook, we collect movie data through the following steps:
- We use the [IMDb Non-commercial dataset](https://developer.imdb.com/non-commercial-datasets/) to select movies released between 2000 and 2024.
- We make API calls to the [OMDB API](https://www.omdbapi.com) to retrieve metadata for the selected movies. Each record includes details such as title, release date, genre, and a URL to the movie poster image.
- We download the movie posters as .jpg files using the provided URLs.

### **1.1 Import necessary libaries**

In [33]:
import os
import pandas as pd
import requests
from PIL import Image 
import pickle
from tqdm import tqdm

from config_loader import load_config_value
openai_api_key = load_config_value("OPENAI_API_KEY")
omdb_key = load_config_value("OMDB_API_KEY")

### **1.2 Import already existing dataset**
This dataset includes movies from 2014 to 2024

In [5]:
df = pd.read_csv('movies_dataset_final.csv')
df.dropna(inplace=True, axis=0)
df = df.reset_index(drop=True)
df.drop('Unnamed: 0', inplace=True, axis=1)

### **1.3 Download additional data**
We need to download additional data, from 2000 to 2013, to have a good representation of all movie genres.

In [21]:
def collect_data_ids(basics_path, ratings_path):
    """
    Filter IMDb datasets to collect movie IDs of interest.

    Args:
        basics_path (str): Path to the `title.basics.tsv` file.
        ratings_path (str): Path to the `title.ratings.tsv` file.
        
    Returns:
        List: A list containing the movie IDs.
    """
    
    
    ### 1. Load and Filter `title.basics.tsv`
    basics = pd.read_csv(basics_path, sep='\t', low_memory=False)
    basics_filtered = basics[
        (basics['titleType'] == 'movie') & 
        (basics['isAdult'] == '0') & 
        (basics['startYear'].apply(lambda x: x.isdigit() and int(x) >= 2000 and int(x) < 2014))
    ]

    ### 2. Load and Filter `title.ratings.tsv`
    ratings = pd.read_csv(ratings_path, sep='\t', low_memory=False)
    ratings_filtered = ratings[ratings['averageRating'] >= 7]

    ### 3. Join Basics and Ratings on `tconst`
    filtered_movies = basics_filtered.merge(ratings_filtered, on='tconst', how='inner')

    ### 4. Get IMDb Movie IDs
    filtered_ids = filtered_movies['tconst'].tolist()
    movie_ids = list(set(filtered_ids))  # Ensure unique IDs

    return movie_ids


In [23]:
basics_path = 'Data/IMDB_Dataset/title.basics.tsv'
ratings_path = 'Data/IMDB_Dataset/title.ratings.tsv'

movie_ids = collect_data_ids(basics_path, ratings_path)

# Save the result to a .txt file
with open('movie_ids_list.txt', 'w+') as f:
    for id in movie_ids:
         f.write('%s\n' %str(id))

f.close()

In [25]:
def fetch_movie_data_from_omdb(movie_ids, api_key, save_path):
    """
    Fetch movie information from the OMDB API based on IMDb movie IDs.
    
    If a pickle file exists, it resumes from where it left off.

    Args:
        movie_ids (list): List of filtered movie IDs.
        api_key (str): Your OMDB API key.
        save_path (str): Path to save progress as a pickle file.

    Returns:
        pd.DataFrame: A DataFrame containing the fetched movie information.
    """
    
    # Try loading previously saved progress
    try:
        with open(save_path, "rb") as f:
            movie_data_list = pickle.load(f)
            fetched_ids = {movie["imdb_id"] for movie in movie_data_list}  # Track already fetched IDs
            print(f"Resuming from {len(movie_data_list)} movies...")
    except (FileNotFoundError, EOFError):
        movie_data_list = []
        fetched_ids = set()

    # Fetch only missing movie IDs
    remaining_ids = [m_id for m_id in movie_ids if m_id not in fetched_ids]

    for movie_id in tqdm(remaining_ids, desc="Fetching movie data"):
        print(f"Fetching data for movie ID {movie_id} from OMDb...")
        
        url = f"http://www.omdbapi.com/?apikey={api_key}&i={movie_id}&plot=full"
        response = requests.get(url)
        
        if response.status_code == 200:
            movie_data = response.json()
            
            if movie_data.get("Response", "False") == "True":
                # Store relevant fields
                movie_data_list.append({
                    "imdb_id": movie_id,
                    "title": movie_data.get("Title", None),
                    "year": movie_data.get("Year", None),
                    "genre": movie_data.get("Genre", None),
                    "director": movie_data.get("Director", None),
                    "actors": movie_data.get("Actors", None),
                    "plot": movie_data.get("Plot", None),
                    "country": movie_data.get("Country", None),
                    "awards": movie_data.get("Awards", None),
                    "poster": movie_data.get("Poster", None),
                    "rating": movie_data.get("imdbRating", None),
                    'metascore':movie_data.get("imdbRating", None),
                    "votes": movie_data.get("Metascore", None),
                    "type": movie_data.get("Type", None)
                })

                # Save progress every 10 movies
                if len(movie_data_list) % 100 == 0:
                    with open(save_path, "wb") as f:
                        pickle.dump(movie_data_list, f)
                    print(f"Progress saved after {len(movie_data_list)} movies.")
        else:
            print(f"Failed to fetch data for ID: {movie_id}, Status Code: {response.status_code}")

    # Final save after completion
    with open(save_path, "wb") as f:
        pickle.dump(movie_data_list, f)
    print("Final dataset saved!")

    # Convert to DataFrame and return
    movie_df = pd.DataFrame(movie_data_list)
    return movie_df

In [39]:
save_path = "movie_data.pkl"
movie_df = fetch_movie_data_from_omdb(movie_ids, omdb_key, save_path)

movie_df.to_csv("filtered_movies.csv", index=False)

Resuming from 22600 movies...


Fetching movie data:   0%|                     | 1/3591 [00:00<07:11,  8.31it/s]

Fetching data for movie ID tt29456494 from OMDb...
Fetching data for movie ID tt2783878 from OMDb...


Fetching movie data:   0%|                     | 2/3591 [00:00<06:45,  8.85it/s]

Fetching data for movie ID tt4164468 from OMDb...
Fetching data for movie ID tt0183659 from OMDb...


...

Fetching data for movie ID tt1584781 from OMDb...
Fetching data for movie ID tt0805604 from OMDb...


Fetching movie data: 100%|█████████████████▉| 3589/3591 [09:32<00:00,  6.18it/s]

Fetching data for movie ID tt1291523 from OMDb...
Fetching data for movie ID tt1224134 from OMDb...


Fetching movie data: 100%|██████████████████| 3591/3591 [09:33<00:00,  6.26it/s]


Fetching data for movie ID tt3112556 from OMDb...
Final dataset saved!


In [41]:
with open(save_path, "rb") as f:
    movie_data_list = pickle.load(f)

# Convert to DataFrame
movie_df = pd.DataFrame(movie_data_list)
movie_df.to_csv("filtered_movies.csv", index=False)

In [54]:
movie_df = movie_df.rename(columns={'imdb_id' : 'tconst'})
movie_df.head()

,tconst,title,year,genre,director,actors,plot,country,awards,poster,rating,metascore,votes,type
0,tt0406933,Life After War,2003,Documentary,Brian Knappenberger,N/A,N/A,USA,N/A,N/A,6.8,6.8,N/A,movie
1,tt0334482,Thenkasipattanam,2000,"Comedy, Romance","Meccartin, Raffi","Suresh Gopi, Lal, Dileep",Kannan is in love with his childhood friend Me...,India,N/A,https://m.media-amazon.com/images/M/MV5BNDUzNz...,7.6,7.6,N/A,movie
2,tt2010896,As I Die Slowly,2011,Drama,"Michael Wayne Thomas, Marcus Damone Henry","Michael Wayne Thomas, Michelle Hays, Ash Davit...","What happens when a man who has it all, become...",USA,N/A,https://m.media-amazon.com/images/M/MV5BMjM2MT...,N/A,N/A,N/A,movie
3,tt0405086,The Lady from Sockholm,2005,"Comedy, Mystery, Crime","Evan Lieberman, Eddy Von Mueller","Chris Clabo, Eric Goins, Melanie Parker",A sock puppet detective unravels a case involv...,United States,N/A,https://m.media-amazon.com/images/M/MV5BMTMxNz...,7.6,7.6,N/A,movie
4,tt0382033,First Person,2003,"Drama, Thriller",Richard D'Angelo,"Keenan Henson, Carey Macaleer, Gregor Xythe",Edward is our host for the documentary on the ...,United States,2 wins,N/A,7.2,7.2,N/A,movie


### **1.4 Concatenate the two datasets**

In [56]:
# Concatenate the dataframes
df_combined = pd.concat([df, movie_df], ignore_index=True)

# Remove duplicates based on 'tconst' (assuming 'tconst' is a unique identifier)
df_combined = df_combined.drop_duplicates(subset="tconst", keep="first")

# Reset index
df_combined.reset_index(drop=True, inplace=True)

In [60]:
df_combined.to_csv("filtered_movies.csv", index=False)

In [ ]:
posters = df['poster'].to_list()

In [ ]:
# for i, url in enumerate(posters):
#     data = requests.get(url).content 
#     # print(df['tconst'][i])
#     # print(i)
#     f = open('Data/' + df['tconst'][i] + '.jpg','wb') 
#     f.write(data) 
#     f.close()